In [3]:
import pandas as pd

translations = pd.read_csv('data/pt-en.tsv', sep='\t', encoding='utf-8', header=None)
translations = translations[[1, 3]].rename(columns={1: 'pt', 3: 'en'})
translations = translations.dropna()

translations = translations[translations['pt'].str.split().apply(len) <= 10]
translations = translations[translations['en'].str.split().apply(len) <= 10]

translations = translations.reindex()

translations.to_csv('data/dataset.tsv', sep='\t')

translations


,pt,en
0,Uma menina chorando abriu a porta.,A crying girl opened the door.
1,Vamos tentar alguma coisa!,Let's try something.
2,Preciso ir dormir.,I have to go to sleep.
3,Preciso ir dormir.,I need to go to sleep.
4,O sinal '&' significa 'e'.,The sign '&' stands for 'and'.
...,...,...
295629,Somos da França.,We're from France.
295630,Nós somos da França.,We're from France.
295631,Tom está apenas se divertindo.,Tom is just having fun.
295632,Ele faleceu? Eu nem sabia que ele estava doente!,He's dead? I didn't even know he was sick!


Just following the step-by-step [tutorial from Tokenizers library](https://huggingface.co/docs/tokenizers/pipeline#all-together-a-bert-tokenizer-from-scratch) for creating a Bert tokenizer on our corpus

In [4]:
from src.tokenizer import BertTokenizer

pt_tokenizer = BertTokenizer()
pt_tokenizer.train_from_iterator(translations['pt'])

en_tokenizer = BertTokenizer()
en_tokenizer.train_from_iterator(translations['en'])

pt_tokenizer.tokenizer.save('data/pt-tokens.json')
en_tokenizer.tokenizer.save('data/en-tokens.json')

In [2]:
from src.tokenizer import BertTokenizer
# Lets read the tokenizer from the files saved
pt_tokenizer = BertTokenizer(path='data/pt-tokens.json')
en_tokenizer = BertTokenizer(path='data/en-tokens.json')

UnboundLocalError: cannot access local variable 'tokenizer' where it is not associated with a value

In [6]:
output = pt_tokenizer.encode("Oi, como vai?")
print(output.ids)

pt_tokenizer.decode(output.ids)

[1, 3166, 14, 313, 317, 32, 2]


'oi, como vai?'

In [7]:
output = en_tokenizer.encode("Hi, how are you?")
print(output.ids)

en_tokenizer.decode(output.ids)

[1, 2596, 16, 272, 207, 152, 34, 2]


'hi, how are you?'